# Python Libraries

In [3]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import datetime
import time
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('max_colwidth', -1)

<ipython-input-2-af59b17b2a7e>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# Importing/Loading of Kaggle Google App Dataset

In [4]:
filepath = "/Users/ketansahu/Documents/DataEngineeringProject/GitHubRepo/googleplaystore.csv"
df = pd.read_csv(filepath ,encoding='utf-8')
col_name = ['app',  'category', 'rating', 'reviews', 'app_size', 'installs', 'app_type', 'price', 'content_rating', 'genres', 'last_updated', 'current_version', 'android_version']
df.columns = col_name
df

,app,category,rating,reviews,app_size,installs,app_type,price,content_rating,genres,last_updated,current_version,android_version
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10835,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10836,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10837,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [5]:
#Add 500 Random user_id
df["user_id"] = np.random.randint(1, 500, len(df))
#Add download date
df["download_date"] = pd.date_range('today', periods=len(df), freq='30min').normalize()
#reindex the column order
df = df.reindex(columns=['download_date', 'user_id', 'app',  'category', 'rating', 'reviews', 'app_size', 'installs', 'app_type', 'price', 'content_rating', 'genres', 'last_updated', 'current_version', 'android_version'])
#Sort the Dataframe based on the the download_date
df = df.sort_values('download_date')
#reset the index=row numbers
df = df.reset_index(drop =True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10839 entries, 0 to 10838
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   download_date    10839 non-null  datetime64[ns]
 1   user_id          10839 non-null  int64         
 2   app              10839 non-null  object        
 3   category         10839 non-null  object        
 4   rating           9365 non-null   float64       
 5   reviews          10839 non-null  int64         
 6   app_size         10839 non-null  object        
 7   installs         10839 non-null  object        
 8   app_type         10838 non-null  object        
 9   price            10839 non-null  object        
 10  content_rating   10839 non-null  object        
 11  genres           10839 non-null  object        
 12  last_updated     10839 non-null  object        
 13  current_version  10831 non-null  object        
 14  android_version  10837 non-null  objec

# App Details

In [6]:
#Assign app_id to each unique app in the dataset. app_id starts from 500 to #
df['app_id'] = df.groupby(['app']).ngroup()
df['app_id'] = df['app_id'].map(lambda q: q+500)

In [7]:
#Total unique app in the data set
df_app = df['app'].unique()
len(df_app)

9659

# Fake User Details Dataset

In [8]:
from faker import Faker
fake = Faker()
n = 500
faker_data = pd.DataFrame([[fake.name(), np.random.randint(18,65), fake.state()] for _ in range(n)],
     columns=['user_name', 'user_age', 'user_location'])
faker_data['user_id'] = range(1, 1+len(faker_data))
#user_id = pd.factorize(faker_data.apply(tuple, axis=1))[0] + 1
#faker_data.insert(loc=0, column='user_id', value=user_id)
faker_data.head(30)

,user_name,user_age,user_location,user_id
0,Bruce Rice,49,New Mexico,1
1,Angela Wells,59,Arkansas,2
2,Edward Hall,47,Alaska,3
3,Jeffrey Andersen,48,Vermont,4
4,Kelly Mays MD,42,Utah,5
5,Thomas Simpson,49,Kansas,6
6,Denise Wood DDS,19,Maryland,7
7,Erik Shaw,62,New York,8
8,Corey Cook,63,Florida,9
9,Christina Richmond,26,Michigan,10


# Data Merge (Kaggle Google App Data + Fake User Details Data)

In [9]:
#Merge Faker data to kaggle data on left
users_app_data = df.merge(faker_data, on='user_id', how='left')
#Add time spent column
users_app_data['time_spent_min'] = np.random.randint(30, 250, len(users_app_data))
users_app_data

,download_date,user_id,app,category,rating,reviews,app_size,installs,app_type,price,content_rating,genres,last_updated,current_version,android_version,app_id,user_name,user_age,user_location,time_spent_min
0,2021-05-07,244,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,7462,Dr. Kelsey Hancock MD,19,Washington,135
1,2021-05-07,120,3D Color Pixel by Number - Sandbox Art Coloring,ART_AND_DESIGN,4.4,1518,37M,"100,000+",Free,0,Everyone,Art & Design,"August 3, 2018",1.2.3,2.3 and up,560,Mark Robinson,36,Arizona,194
2,2021-05-07,3,Mandala Coloring Book,ART_AND_DESIGN,4.6,4326,21M,"100,000+",Free,0,Everyone,Art & Design,"June 26, 2018",1.0.4,4.4 and up,6525,Edward Hall,47,Alaska,75
3,2021-05-07,173,Name Art Photo Editor - Focus n Filters,ART_AND_DESIGN,4.4,8788,12M,"1,000,000+",Free,0,Everyone,Art & Design,"July 31, 2018",1.0.15,4.0 and up,7023,Jacqueline Robinson,49,Maryland,185
4,2021-05-07,402,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,28M,"1,000,000+",Free,0,Everyone,Art & Design,"October 27, 2017",1.0.4,4.1 and up,8832,Angela Edwards,51,Nebraska,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,2021-12-19,123,Fanfic-FR,BOOKS_AND_REFERENCE,3.3,52,3.6M,"5,000+",Free,0,Teen,Books & Reference,"August 5, 2017",0.3.4,4.1 and up,4868,Meredith Galloway,55,Rhode Island,172
10835,2021-12-19,429,Gold Quote - Gold.fr,FINANCE,NaN,96,1.5M,"10,000+",Free,0,Everyone,Finance,"May 19, 2016",2.3,2.2 and up,5388,Jeffrey Smith,48,Delaware,230
10836,2021-12-19,283,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device,8894,Amanda Martin,27,Missouri,58
10837,2021-12-19,235,Fr Agnel Ambarnath,FAMILY,4.2,117,13M,"5,000+",Free,0,Everyone,Education,"June 13, 2018",2.0.20,4.0.3 and up,5103,Crystal George,28,Hawaii,111


# Data Cleaning

In [10]:
#Check Total Duplicates
(users_app_data.duplicated().value_counts()/users_app_data.shape[0])*100

False    100.0
dtype: float64

In [11]:
# Fill Nan in rating data
rating_data=users_app_data[pd.notnull(users_app_data['rating'])]
median=np.median(rating_data['rating'])
users_app_data['rating'].fillna(median,inplace=True)

In [11]:
# from datetime import date
# today = date.today()

# # dd/mm/YY
# d1 = today.strftime("%Y-%m-%d")
# df = users_app_data[users_app_data.download_date == str(today.strftime("%Y/%m/%d"))]


In [12]:
users_app_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10839 entries, 0 to 10838
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   download_date    10839 non-null  datetime64[ns]
 1   user_id          10839 non-null  int64         
 2   app              10839 non-null  object        
 3   category         10839 non-null  object        
 4   rating           10839 non-null  float64       
 5   reviews          10839 non-null  int64         
 6   app_size         10839 non-null  object        
 7   installs         10839 non-null  object        
 8   app_type         10838 non-null  object        
 9   price            10839 non-null  object        
 10  content_rating   10839 non-null  object        
 11  genres           10839 non-null  object        
 12  last_updated     10839 non-null  object        
 13  current_version  10831 non-null  object        
 14  android_version  10837 non-null  objec

In [13]:
users_app_data= users_app_data[0:1000]
users_app_data.index.names = ['event_id']

users_app_data.to_csv('users_app_batch_client_data.csv')